In [1]:
from bert.extract_feature import BertVector
bv = BertVector()

I:BERT_VEC:[graph:opt: 33]:model config: /Users/Paul/Documents/GitHub/FakeNewsDetector/bert/chinese_L-12_H-768_A-12/bert_config.json
I:BERT_VEC:[graph:opt: 39]:build graph...
I:BERT_VEC:[graph:opt: 92]:load parameters from checkpoint...
I:BERT_VEC:[graph:opt: 94]:freeze...
INFO:tensorflow:Froze 181 variables.
INFO:tensorflow:Converted 181 variables to const ops.
I:BERT_VEC:[graph:opt: 97]:optimize...
I:BERT_VEC:[graph:opt:106]:write graph to a tmp file: /Users/Paul/Documents/GitHub/FakeNewsDetector/bert/chinese_L-12_H-768_A-12/../tmp/result/graph
INFO:tensorflow:Using config: {'_model_dir': '../tmp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
  allow_growth: true
}
graph_options {
  optimizer_options {
    global_jit_level: ON_1
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_d

In [2]:
import pickle
import pandas as pd

data = pd.read_pickle("./data_0104.pkl")
print("Shape of data", data.shape) 
print("Number of rows : ", data.shape[0]) 
print("Number of columns : ", data.shape[1]) 

Shape of data (4322, 6)
Number of rows :  4322
Number of columns :  6


In [3]:
target_feature = 'title'
data_set = data.filter([target_feature, 'label'])

In [4]:
print(data_set.head(10))

                                         title  label
0                    口罩哪裡買 和敗口罩兩個官方帳號已經確認是釣魚帳號      1
1                         皮蛋是用馬尿製成的 含鉛量多吃了還會致癌      1
2  全面屍控免費官方貼圖 - 喪屍來啦！ line://shop/detail/19504      1
3                         吃一包臺灣泡麵 ，就需肝臟解毒32天這謠      1
4               因為當夜幕降臨時，人體系統變成酸性，體質的偵探能力也會著降低      1
5                             泡麵含有防腐劑 吃太多會變木乃伊      1
6                          挑男人沒別的，要挑疼你的 ！這小S說的      1
7                         眼前的字不是字 是眼睛出問題還是心理有問      1
8                       習近平夫人對高考分數發言 ，震驚整個國家教育      1
9                    這是大賣場在打烊以後可能的景象 網路影片別急著轉發      1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [5]:
data_set_filtered = data_set[data_set[target_feature].notnull()]

In [6]:
import re
for idx in range(0, data_set_filtered.shape[0]):
    data_set_filtered[target_feature].iloc[idx] = re.sub(r"[\n\t\s]*", "", data_set_filtered[target_feature].iloc[idx])

In [7]:
data_set_filtered.insert(1, target_feature + "_vector", 0) 
data_set_filtered[target_feature + "_vector"] = data_set_filtered[target_feature + "_vector"].astype(object)
print(data_set_filtered.isnull().any())
print(data_set_filtered.shape)

title           False
title_vector    False
label           False
dtype: bool
(4322, 3)


In [8]:
max_len = 0
for title in data_set_filtered['title']:
    if(len(title) > max_len):
        max_len = len(title)
        print(max_len, title)
max_len

24 口罩哪裡買和敗口罩兩個官方帳號已經確認是釣魚帳號
40 全面屍控免費官方貼圖-喪屍來啦！line://shop/detail/19504
57 湯若望在明朝崇禎年間來到中國傳揚天主教，他精通天文、地理、數學、醫學當時中國的曆法已經暴露出諸多弊端，存在眾多偏差
84 [集！在線等]新年集點活動開跑啦🎉活動頁任選集章換LINEPOINTS，200萬點換完為止！>https://lin.ee/EmbTx6I/crtw/AL/digest
92 5家Costco、7家Target疫情大爆发上百员工确诊!城堡奥莱人挤人或成超级引爆点https://mp.weixin.qq.com/s/9IYWZVHEtWiJtK51I_fSgA
106 平時可以用優酪乳洗澡嗎,現在越來越多的女性都開始重視與個人的皮膚狀態要想有個好的皮膚，平時的保養也是非常關鍵的就有部份女性會選擇洗牛奶浴，來使皮膚更加的白皙有活力那麼就有人會問，如果可以用優酪乳洗澡嗎效果會不會比
271 民進黨在1986年成立之後短短三、五年內，事實上就已充滿蚊子、蒼蠅與蟑螂，開始效法舊國民黨的腐敗，貪婪程度更是青出於藍；以光速般的速度腐爛，但卻學到一身政治操弄與選舉致勝的高超本領，造謠抹黑，無惡不作，行事不擇手段；擅長設定議題，挑撥階級對立與族群仇恨，以捍衛民主自由與弱勢正義之名，行撈錢奪權之實尤有甚者，打從大約1998年開始，更是在國民黨黑金教父李登輝的大力合作與推動下(包括更早之前，大約八零年代末，由李登輝提供民進黨鉅額金錢以發展所謂本土路線)，展開最厲害的一項政治操弄法寶之戰略定位，亦即仇中反華；名為「愛台灣」，實則挑撥族群仇恨


271

In [9]:
import sys

for idx in range(0, data_set_filtered.shape[0]):
    sys.stdout.write('\r'+ "Implementing BERT vectoring {}% \n".format(round(100 * idx/data_set_filtered.shape[0], 2)))
    data_set_filtered[target_feature + "_vector"].iloc[idx] = bv.encode([data_set_filtered[target_feature].iloc[idx]])

身 [SEP]
INFO:tensorflow:input_ids: 101 7344 6011 2166 3791 8038 4017 7937 3779 3188 4197 6857 7938 1962 4500 2218 6275 6275 4692 8024 1162 6366 974 3175 5183 1927 1001 749 8024 2215 1071 679 1003 6716 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
Implementing BERT vectoring 99.61% 
INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 0
INFO:tensorflow:tokens: [CLS] 越 南 養 殖 的 巴 沙 魚 ， 台 灣 進 口 很 多 ， 但 這 種 魚 養 殖 在 湄 公 河 極 度 污 染 的 水 域 ， 不 但 含 重 金 

In [10]:
print(data_set_filtered.shape)

(4322, 3)


In [11]:
data_set_filtered.to_pickle('data.pkl')

In [12]:
del data_set_filtered